# Data Preprocessing

In [7]:
#import dependencies
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

#set up connection to databse
from sqlalchemy import create_engine


protocol = 'postgresql'
username = 'postgres'
password = 'administrator'
host = 'wine.corp16oy9wyq.us-east-1.rds.amazonaws.com'
port = 5432
database = 'wine'

rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database}'
        
engine = create_engine(rds_connection_string)
conn = engine.connect()

In [8]:
#Retrieve Data from RDS SQL Database
data = pd.read_sql("SELECT * FROM wine_data", conn)
data.head()

,wine_id,country,description,designation,points,price,province,region_1,region_2,taster_name,...,max_temp,precip,humidity,heat_index,latitude_y,longitude_y,elevation,review_neg,review_neu,review_pos
0,0,US,"Tart and snappy, the flavors of lime flesh and...",None,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,114.0,37.43,70.76,109.6,45.505794,-122.781770,126.0,0.053,0.947,0.000
1,1,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.805,0.195
2,2,US,This bright and spicy wine refreshes with pala...,None,90,16.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.909,0.091
3,3,US,"Full-bodied and forward, this offers fruit fla...",Vintner's Reserve,87,45.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.127,0.873,0.000
4,4,US,This estate-grown Pinot is still pretty rough ...,None,85,25.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.894,0.106


In [9]:
print(data.nunique())


wine_id                  49780
country                      1
description              46080
designation              12971
points                      21
price                      158
province                     4
region_1                   202
region_2                    17
taster_name                 12
taster_twitter_handle       11
title                    45892
variety                    243
winery_name               4917
latitude_x                3563
longitude_x               3563
min_temp                   439
max_temp                   287
precip                    1763
humidity                  1428
heat_index                 261
latitude_y                3563
longitude_y               3563
elevation                  623
review_neg                 256
review_neu                 477
review_pos                 467
dtype: int64


In [10]:
data.drop(["wine_id", "country", "description", "designation","taster_name", "taster_twitter_handle", "title" ], axis=1, inplace=True)
data.head()

,points,price,province,region_1,region_2,variety,winery_name,latitude_x,longitude_x,min_temp,max_temp,precip,humidity,heat_index,latitude_y,longitude_y,elevation,review_neg,review_neu,review_pos
0,87,14.0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Rainstorm winery,45.505794,-122.781770,20.8,114.0,37.43,70.76,109.6,45.505794,-122.781770,126.0,0.053,0.947,0.000
1,87,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks winery,43.956731,-123.279126,20.2,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.805,0.195
2,90,16.0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Sweet Cheeks winery,43.956731,-123.279126,20.2,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.909,0.091
3,87,45.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks winery,43.956731,-123.279126,20.2,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.127,0.873,0.000
4,85,25.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks winery,43.956731,-123.279126,20.2,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.894,0.106


In [15]:
#create bins for price ranges
def price_to_range(price): 
  if (price < 20): 
    return 0 
  elif (price >20) and (price <= 45):
    return 1
  if (price >45) and (price <= 80):
    return 2 
  if (price >80) and (price <= 100):
    return 3
  if (price >140) and (price <= 300): 
    return 4
  else: 
    return 5
  
data["price"] = data["price"].apply(price_to_range)
data = data.rename(columns={"price": "price_range"})
data.head()

,points,price_range,province,region_1,region_2,variety,winery_name,latitude_x,longitude_x,min_temp,max_temp,precip,humidity,heat_index,latitude_y,longitude_y,elevation,review_neg,review_neu,review_pos
0,87,0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Rainstorm winery,45.505794,-122.781770,20.8,114.0,37.43,70.76,109.6,45.505794,-122.781770,126.0,0.053,0.947,0.000
1,87,2,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks winery,43.956731,-123.279126,20.2,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.805,0.195
2,90,0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Sweet Cheeks winery,43.956731,-123.279126,20.2,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.909,0.091
3,87,1,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks winery,43.956731,-123.279126,20.2,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.127,0.873,0.000
4,85,1,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks winery,43.956731,-123.279126,20.2,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.894,0.106


In [16]:
def points_to_range(points): 
  if (points >= 90):
    return 0
  else: 
    return 1
  
data["points"] = data["points"].apply(points_to_range)
data = data.rename(columns={"points": "point_range"})
data.head()

,point_range,price_range,province,region_1,region_2,variety,winery_name,latitude_x,longitude_x,min_temp,max_temp,precip,humidity,heat_index,latitude_y,longitude_y,elevation,review_neg,review_neu,review_pos
0,1,0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Rainstorm winery,45.505794,-122.781770,20.8,114.0,37.43,70.76,109.6,45.505794,-122.781770,126.0,0.053,0.947,0.000
1,1,2,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks winery,43.956731,-123.279126,20.2,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.805,0.195
2,0,0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Sweet Cheeks winery,43.956731,-123.279126,20.2,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.909,0.091
3,1,1,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks winery,43.956731,-123.279126,20.2,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.127,0.873,0.000
4,1,1,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Sweet Cheeks winery,43.956731,-123.279126,20.2,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.894,0.106


In [22]:
variety_values = data["variety"].value_counts()
variety_value_counts = variety_values[variety_values > 100]
variety_value_counts

Pinot Noir                  8914
Cabernet Sauvignon          6965
Chardonnay                  6325
Syrah                       3083
Red Blend                   2645
Zinfandel                   2532
Merlot                      2178
Sauvignon Blanc             1916
Bordeaux-style Red Blend    1696
Riesling                    1587
Cabernet Franc               844
Rosé                         804
Pinot Gris                   770
Rhône-style Red Blend        666
Viognier                     660
Petite Sirah                 656
Sparkling Blend              615
White Blend                  543
Grenache                     503
Malbec                       499
Sangiovese                   384
Gewürztraminer               337
Pinot Grigio                 335
Tempranillo                  307
Meritage                     219
Rhône-style White Blend      212
Barbera                      211
Mourvèdre                    196
Petit Verdot                 179
Pinot Blanc                  178
G-S-M     

In [24]:
# Choose a cutoff value and create a list of varieties to be replaced
cutoff = 1500
variety_types_to_replace = list(variety_values[variety_values <= cutoff].index)

# Replace in dataframe
for variety in variety_types_to_replace:
   data['variety'] = data['variety'].replace(variety,"Other")

# Check to make sure binning was successful
data['variety'].value_counts()

Other                       11939
Pinot Noir                   8914
Cabernet Sauvignon           6965
Chardonnay                   6325
Syrah                        3083
Red Blend                    2645
Zinfandel                    2532
Merlot                       2178
Sauvignon Blanc              1916
Bordeaux-style Red Blend     1696
Riesling                     1587
Name: variety, dtype: int64

In [29]:
precip = data['precip'].value_counts()
precip[precip > 10]

23.01    636
16.60    479
7.65     456
10.60    404
10.85    371
        ... 
27.75     11
16.18     11
39.18     11
31.55     11
10.53     11
Name: precip, Length: 934, dtype: int64

In [30]:
print(data.nunique())


point_range       2
price_range       6
province          4
region_1        202
region_2         17
variety          11
winery_name    4917
latitude_x     3563
longitude_x    3563
min_temp        439
max_temp        287
precip         1763
humidity       1428
heat_index      261
latitude_y     3563
longitude_y    3563
elevation       623
review_neg      256
review_neu      477
review_pos      467
dtype: int64


# Machine Learning

# Saving Model